## Imports

In [1]:
# pip install pgeocode

In [2]:
import pandas as pd
import pgeocode
import numpy as np

## Helper methods

In [3]:
def missing_values_counts(df: pd.DataFrame) -> pd.Series:
    """Returns number of missing values in each column"""
    return df.isnull().sum()

def missing_values_percent(df: pd.DataFrame) -> pd.Series:
    """Returns percentage of missing values in each column"""
    percent_null = (df.isnull().sum() / df.shape[0]) * 100
    return pd.Series(percent_null, index=df.columns)

def remove_null_columns(df: pd.DataFrame, limit=50, zero_is_null=True) -> pd.DataFrame:
    """Drops all columns with more than limit percent null values"""
    rows  = df.shape[0]
    cols  = df.columns
    drops = []
    
    percent_null = lambda c: (df[c].isnull().sum() / rows) * 100
    is_zero      = lambda v: True if v == 0 else False
    percent_zero = lambda c: (df[c].apply(is_zero).sum() / rows) * 100
    
    for col in cols:
        if percent_null(col) > limit:
            drops.append(col)
    
        if (zero_is_null) and (percent_zero(col) > limit):
            drops.append(col)

    return df.drop(drops, 1)

def lower_case_data(df: pd.DataFrame) -> pd.DataFrame:
    cols = df.select_dtypes(include=['object', 'category']).columns
    
    for col in cols:
        df[col] = df[col].str.lower()
    
    return df



## Reading Data

In [4]:
# customer_path = "data/customer.csv"
# inventory_path = "data/inventory.csv"
invoice_path = "data/invoice.csv"

# customer = pd.read_csv(customer_path)
# inventory = pd.read_csv(inventory_path)
invoice = pd.read_csv(invoice_path)

# print(customer.shape)
# print(inventory.shape)
print(invoice.shape)

(492314, 59)


## Converting all text data to lower case 

In [5]:
invoice = lower_case_data(invoice)
invoice.head()

,Unnamed: 0,Amt Rcvd From Custom,Amt Rcvd From Ins Co,Area / Locality,CGST(14%),CGST(2.5%),CGST(6%),CGST(9%),CITY,Cash /Cashless Type,...,Service Advisor Name,TDS amount,Technician Name,Total Amt Wtd Tax.,Total CGST,Total GST,Total IGST,Total SGST/UGST,Total Value,User ID
0,0,0.0,0.0,majiwada,0.0,0.0,0.0,0.0,thane,NaN,...,NaN,0.0,rupesh,4051.95,0.0,0.0,0.0,0.0,4051.95,bc01fs1
1,1,0.0,0.0,thnae,0.0,0.0,0.0,0.0,thnae,NaN,...,NaN,0.0,prashant,1001.29,0.0,0.0,0.0,0.0,1001.29,bc01sa2
2,2,0.0,0.0,thane,0.0,0.0,0.0,0.0,thane[w],NaN,...,NaN,0.0,imran,233.14,0.0,0.0,0.0,0.0,233.14,bc01sa2
3,3,0.0,0.0,thnae,0.0,0.0,0.0,0.0,thnae,NaN,...,NaN,0.0,prashant,0.00,0.0,0.0,0.0,0.0,0.00,bc01sa2
4,4,0.0,0.0,thane,0.0,0.0,0.0,0.0,thane,NaN,...,NaN,0.0,dayanand,2747.12,0.0,0.0,0.0,0.0,2747.12,bc01sa2


## Dropping colums with more than 40% null values

In [6]:
max_percent = 40

# customer = remove_null_columns(customer, limit=max_percent)
# inventory = remove_null_columns(inventory, limit=max_percent)
invoice = remove_null_columns(invoice, limit=max_percent)

# print(customer.shape)
# print(inventory.shape)
print(invoice.shape)

(492314, 25)


In [7]:
invoice.columns

Index(['Unnamed: 0', 'Area / Locality', 'CITY', 'Cust Type', 'Customer No.',
       'District', 'Gate Pass Time', 'Invoice Date', 'Invoice No',
       'Invoice Time', 'Job Card No', 'JobCard Date', 'JobCard Time',
       'KMs Reading', 'Labour Total', 'Make', 'Model', 'Order Type',
       'Pin code', 'Plant', 'Plant Name1', 'Print Status', 'Regn No',
       'Total Amt Wtd Tax.', 'User ID'],
      dtype='object')

## Dropping unnecessary columns

In [8]:
# inventory = inventory.drop(['Unnamed: 0'], 1)
invoice = invoice.drop(['Unnamed: 0', 'Print Status',], 1)
invoice.columns


Index(['Area / Locality', 'CITY', 'Cust Type', 'Customer No.', 'District',
       'Gate Pass Time', 'Invoice Date', 'Invoice No', 'Invoice Time',
       'Job Card No', 'JobCard Date', 'JobCard Time', 'KMs Reading',
       'Labour Total', 'Make', 'Model', 'Order Type', 'Pin code', 'Plant',
       'Plant Name1', 'Regn No', 'Total Amt Wtd Tax.', 'User ID'],
      dtype='object')

In [9]:
invoice.shape

(492314, 23)

## Renaming columns to python friendly names

In [10]:
columns = {
    'Business Partner': 'business_partner', 
    'Customer No.': 'customer_no',
    'Partner Type': 'partner_type',
    'Data Origin': 'data_origin',
    'Title': 'title'
    }

# customer = customer.rename(columns=columns)
# customer.columns

In [11]:
columns = {
    'DBM Order': 'job_card_no', 
    'Order Item': 'order_item',
    'Material': 'material',
    'Description': 'description',
    'Item Category': 'item_category',
    'Order Quantity': 'order_quantity',
    'Target quantity UoM': 'target_quantity_unit',
    'Net value': 'net_value'
    }

# inventory = inventory.rename(columns=columns)
# inventory.columns

In [12]:
columns = {
    'Area / Locality': 'locality',
    'CITY': 'city',
    'Cust Type': 'customer_type',
    'Customer No.': 'customer_no',
    'District': 'district',
    'Gate Pass Time': 'gate_pass_time', 
    'Invoice Date': 'invoice_date',
    'Invoice No': 'invoice_no' ,
    'Invoice Time': 'invoice_time',
    'Job Card No': 'job_card_no',
    'JobCard Date': 'job_card_date',
    'JobCard Time': 'job_card_time',
    'KMs Reading': 'km_reading',
    'Labour Total': 'labour_total',
    'Make': 'make',
    'Model': 'model',
    'ODN No.': 'odn_no',
    'Order Type': 'order_type',
    'Parts Total': 'parts_total',
    'Pin code': 'pin_code',
    'Plant': 'plant_code',
    'Plant Name1': 'plant_name',
    'Print Status': 'print_status',
    'Regn No': 'regn_no',
    'Technician Name': 'technician_name',
    'Total Amt Wtd Tax.': 'total_amt',
    'Total Value': 'total_value',
    'User ID': 'user_id'
    }

invoice = invoice.rename(columns=columns)
invoice.columns

Index(['locality', 'city', 'customer_type', 'customer_no', 'district',
       'gate_pass_time', 'invoice_date', 'invoice_no', 'invoice_time',
       'job_card_no', 'job_card_date', 'job_card_time', 'km_reading',
       'labour_total', 'make', 'model', 'order_type', 'pin_code', 'plant_code',
       'plant_name', 'regn_no', 'total_amt', 'user_id'],
      dtype='object')

## Getting location data from pincode using pgeocode lib

In [13]:

country = pgeocode.Nominatim("in")

invoice["pin_city"]=country.query_postal_code(np.array(invoice["pin_code"].astype(str))).county_name
invoice["pin_district"]=country.query_postal_code(np.array(invoice["pin_code"].astype(str))).state_name
invoice["pin_location"]=country.query_postal_code(np.array(invoice["pin_code"].astype(str))).place_name
invoice["latitude"]=country.query_postal_code(np.array(invoice["pin_code"].astype(str))).latitude
invoice["longitude"]=country.query_postal_code(np.array(invoice["pin_code"].astype(str))).longitude


In [14]:
invoice[["pin_city", "pin_district", "pin_location", "latitude", "longitude"]]


,pin_city,pin_district,pin_location,latitude,longitude
0,Thane,Maharashtra,"Thane Bazar, Thane H.O, Thane R.S.",19.1941,73.000200
1,Thane,Maharashtra,"Thane East, Kopri Colony",19.1941,73.000200
2,Thane,Maharashtra,"Chitalsar Manpada, Sandozbaugh",19.1941,73.000200
3,Thane,Maharashtra,"Thane East, Kopri Colony",19.1941,73.000200
4,Thane,Maharashtra,"Thane East, Kopri Colony",19.1941,73.000200
...,...,...,...,...,...
492309,Tiruvallur,Tamil Nadu,"Ambattur Indl Estate, Athipattu",13.0252,79.856200
492310,Kanchipuram,Tamil Nadu,"Adambakkam, Nilamangai Nagar",12.9918,80.203500
492311,Kanchipuram,Tamil Nadu,"Lakshmipuram, Chromepet, Nehrunagar, Nagalkeni...",12.6668,80.059112
492312,Chennai,Tamil Nadu,Velacheri,13.0647,80.252300


## Imputing missing values with mode

In [15]:
def impute_with_least(df, col):
    least = invoice[col].value_counts().index[-1]
    df[col].fillna(least, inplace=True)

def impute_forwar_fill(df, col):
    df[col].fillna(method="ffill", inplace=True)


In [17]:
for col in ["pin_city", "pin_district", "pin_location", "latitude", "longitude"]:
    impute_forwar_fill(invoice, col)

In [23]:
missing_values_counts(invoice)

customer_type        0
customer_no          0
gate_pass_time       0
invoice_date         0
invoice_no           0
invoice_time         0
job_card_no          0
job_card_date        0
job_card_time        0
km_reading           0
labour_total         0
make                 0
model             1525
order_type           0
pin_code             0
plant_code           0
plant_name           0
regn_no              0
total_amt            0
user_id              0
pin_city             0
pin_district         0
pin_location         0
latitude             0
longitude            0
dtype: int64

In [22]:
invoice.drop(['locality', "district", "city"], 1, inplace=True)

In [24]:
path = "data/clean_invoice.csv"
invoice.to_csv(path, index=False, encoding='utf-8')

In [19]:
regn_mode = invoice.regn_no.mode()[0]
invoice.regn_no.fillna(regn_mode, inplace=True)

In [16]:
missing_values_counts(invoice[["regn_no"]])

regn_no    0
dtype: int64

In [21]:
missing_values_percent(invoice[["pin_city", "pin_district", "pin_location", "latitude", "longitude"]])

pin_city        0.0
pin_district    0.0
pin_location    0.0
latitude        0.0
longitude       0.0
dtype: float64

In [18]:
def impute_with_mode(df:pd.DataFrame, columns:list) -> pd.Series:
    
    for col in columns:
        mode = df.col.mode()[0]
        df.col = df.col.fillna(mode)
    
    return df

In [19]:
path = "data/rto_data.csv"
rto = pd.read_csv(path)

rto.head()


,rto,office_type,office_location,state_code,state_name
0,cg04,rto,raipur,cg,chattisgarh
1,cg10,rto,bilashpur,cg,chattisgarh
2,cg17,rto,jagdalpur,cg,chattisgarh
3,dd02,rto,diu,dd,daman and diu
4,dd03,rto,daman,dd,daman and diu


In [20]:
import re

def get_rto_code(num:str) -> str:
    pat = re.compile(r"^[a-z]{2}[0-9]{1,2}")
    match = re.match(pat, (num))

    if match:
        return match.group()

# x = get_rto_code(invoice.regn_no[0])
# print(x)

In [21]:
invoice["rto_code"] = invoice.regn_no.apply(get_rto_code)
invoice.rto_code

0         ka19
1         mh43
2         ap09
3         mh43
4         mh04
          ... 
492309    tn38
492310    tn22
492311    tn07
492312    tn07
492313    tn10
Name: rto_code, Length: 492314, dtype: object

In [22]:
least = invoice.rto_code.value_counts().index[-1]
invoice.rto_code.fillna(least, inplace=True)
invoice[["rto_code"]].isnull().sum()

rto_code    0
dtype: int64

In [60]:
# invoice[invoice.rto_code == rto]

In [64]:
def get_city_from_rto(rto):
    result = invoice[invoice.rto_code == rto]
    return result['city'].value_counts().index[0]

get_city_from_rto("mh04")

'thane'

In [41]:


invoice.pin_city.apply(get_city_from_rto)

In [44]:
# rows_with_nan = [index for index, row in invoice.iterrows() if row.isnull().any()]

# for val in invoice.pin_city:
#     if val.isna():
#         print(type(val))

import numpy as np

# df = invoice
# index = df["pin_city"].index[df['pin_city'].apply(np.isnan)]


indices = df.loc[pd.isna(df["pin_city"]), :].index

indices


Int64Index([    25,     29,     85,     97,    106,    107,    116,    120,
               142,    145,
            ...
            492061, 492066, 492100, 492127, 492144, 492150, 492161, 492168,
            492262, 492308],
           dtype='int64', length=47246)

In [62]:
# for i in indices:
#     invoice.pin_city[i] = get_city_from_rto(invoice.rto_code[i])

In [65]:
invoice.reset_index(drop=True)

invoice[['pin_city']].apply(lambda x: get_city_from_rto(x) if np.all(pd.isnull(x)) else x, axis=1)

ValueError: Can only compare identically-labeled Series objects

In [30]:
def get_model_from_make(make):
    # if not invoice['model'].notna():
    #     return

    result = invoice[invoice['make']==make]
    return result['model'].value_counts().index[0]

get_model_from_make("honda")

'city'

In [135]:
invoice.make.apply(get_model_from_make)

In [32]:
invoice[["pin_city", "rto_code"]].isnull().sum()

pin_city    47246
rto_code     1053
dtype: int64

In [33]:
missing_values_counts(invoice)

locality          23890
city                  1
customer_type         0
customer_no           0
district              0
gate_pass_time        0
invoice_date          0
invoice_no            0
invoice_time          0
job_card_no           0
job_card_date         0
job_card_time         0
km_reading            0
labour_total          0
make                  0
model              1525
order_type            0
pin_code              0
plant_code            0
plant_name            0
regn_no               0
total_amt             0
user_id               0
pin_city          47246
pin_district      47246
pin_location      47246
latitude          47246
longitude         47246
rto_code           1053
dtype: int64